In [6]:
import allennlp
import allennlp_models
import re 
import string 
import nltk 
import spacy 
import pandas as pd 
import numpy as np 
import math 
from tqdm import tqdm 

from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy

pd.set_option('display.max_colwidth', 200)

# load spaCy model
nlp = spacy.load("en_core_web_sm")

In [7]:
# text = 'Anchor can create a scheduled task for persistence.'
# text = "APT33 has created a scheduled task execute a .vbe file multiple times a day."
text = 'Cobalt Strike can execute a payload on a remote host with PowerShell. This technique does not write any data to disk. Cobalt Strike can also use PowerSploit and other scripting frameworks to perform execution.'

# Plot the dependency graph 
doc = nlp(text) 
displacy.render(doc, style='dep',jupyter=True)

In [ ]:
from nltk.tree import Tree
from IPython.display import display

def nltk_spacy_tree(sent):
    """
    Visualize the SpaCy dependency tree with nltk.tree
    """
    doc = nlp(sent)
    def token_format(token):
        return "_".join([token.orth_, token.tag_, token.dep_])

    def to_nltk_tree(node):
        if node.n_lefts + node.n_rights > 0:
            return Tree(token_format(node),
                       [to_nltk_tree(child)
                        for child in node.children]
                   )
        else:
            return token_format(node)

    tree = [to_nltk_tree(sent.root) for sent in doc.sents]
    # The first item in the list is the full tree
    tree[0].draw()
    # display(tree)

nltk_spacy_tree(text)